<a href="https://colab.research.google.com/github/KevKibe/Hospital-LOS-Prediction/blob/main/HospitalLOSpred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, KFold

from sklearn.linear_model import LinearRegression, Ridge, Lasso

from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestRegressor,BaggingRegressor

from sklearn.preprocessing import LabelEncoder

# For tuning the model

from sklearn.model_selection import GridSearchCV

# To check the model performance

from sklearn.metrics import make_scorer,mean_squared_error, r2_score, mean_absolute_error

In [3]:
#loading the dataset to a dataframe
df= pd.read_csv('/content/healthcare_data (1).csv')
#data exploration 
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 15 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Available Extra Rooms in Hospital  500000 non-null  int64  
 1   Department                         500000 non-null  object 
 2   Ward_Facility_Code                 500000 non-null  object 
 3   doctor_name                        500000 non-null  object 
 4   staff_available                    500000 non-null  int64  
 5   patientid                          500000 non-null  int64  
 6   Age                                500000 non-null  object 
 7   gender                             500000 non-null  object 
 8   Type of Admission                  500000 non-null  object 
 9   Severity of Illness                500000 non-null  object 
 10  health_conditions                  500000 non-null  object 
 11  Visitors with Patient              5000

,Available Extra Rooms in Hospital,Department,Ward_Facility_Code,doctor_name,staff_available,patientid,Age,gender,Type of Admission,Severity of Illness,health_conditions,Visitors with Patient,Insurance,Admission_Deposit,Stay (in days)
0,4,gynecology,D,Dr Sophia,0,33070,41-50,Female,Trauma,Extreme,Diabetes,4,Yes,2966.408696,8
1,4,gynecology,B,Dr Sophia,2,34808,31-40,Female,Trauma,Minor,Heart disease,2,No,3554.835677,9
2,2,gynecology,B,Dr Sophia,8,44577,21-30,Female,Trauma,Extreme,Diabetes,2,Yes,5624.733654,7
3,4,gynecology,D,Dr Olivia,7,3695,31-40,Female,Urgent,Moderate,None,4,No,4814.149231,8
4,2,anesthesia,E,Dr Mark,10,108956,71-80,Male,Trauma,Moderate,Diabetes,2,No,5169.269637,34


In [ ]:
#creating a copy of the data so as not to make changes to the original
same_data= df.copy()

In [ ]:
#checking for duplicate values
df.duplicated().sum()

0

# Data Preparation for Model Building

In [4]:
df = pd.get_dummies(
    df,
    columns = df.select_dtypes(include = ["object", "category"]).columns.tolist(),
    drop_first = True,
)

In [5]:
df.head()

,Available Extra Rooms in Hospital,staff_available,patientid,Visitors with Patient,Admission_Deposit,Stay (in days),Department_anesthesia,Department_gynecology,Department_radiotherapy,Department_surgery,...,Type of Admission_Trauma,Type of Admission_Urgent,Severity of Illness_Minor,Severity of Illness_Moderate,health_conditions_Diabetes,health_conditions_Heart disease,health_conditions_High Blood Pressure,health_conditions_None,health_conditions_Other,Insurance_Yes
0,4,0,33070,4,2966.408696,8,0,1,0,0,...,1,0,0,0,1,0,0,0,0,1
1,4,2,34808,2,3554.835677,9,0,1,0,0,...,1,0,1,0,0,1,0,0,0,0
2,2,8,44577,2,5624.733654,7,0,1,0,0,...,1,0,0,0,1,0,0,0,0,1
3,4,7,3695,4,4814.149231,8,0,1,0,0,...,0,1,0,1,0,0,0,1,0,0
4,2,10,108956,2,5169.269637,34,1,0,0,0,...,1,0,0,1,1,0,0,0,0,0


In [7]:
#correlation analysis
correlations = df.corr()['Stay (in days)']
correlations

Available Extra Rooms in Hospital       -0.019219
staff_available                          0.007398
patientid                                0.006169
Visitors with Patient                    0.027302
Admission_Deposit                        0.044203
Stay (in days)                           1.000000
Department_anesthesia                    0.596323
Department_gynecology                   -0.701090
Department_radiotherapy                  0.217235
Department_surgery                       0.236350
Ward_Facility_Code_B                    -0.240791
Ward_Facility_Code_C                     0.171404
Ward_Facility_Code_D                    -0.263913
Ward_Facility_Code_E                     0.450167
Ward_Facility_Code_F                    -0.268901
doctor_name_Dr John                      0.336819
doctor_name_Dr Mark                      0.510740
doctor_name_Dr Nathan                   -0.193380
doctor_name_Dr Olivia                   -0.234180
doctor_name_Dr Sam                       0.171028


In [15]:
#dropping less correlated features
df = df.drop(['patientid'],axis = 1)

In [21]:
#seperating the feature variables and target variables
X = df.drop('Stay (in days)',axis = 1)

y = df['Stay (in days)'] 

In [22]:
#splitting the training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True, random_state = 1)

#checking the shape of the train and test set
print('Train Shape:',X_train.shape)
print('Test Shape:',X_test.shape)

Train Shape: (400000, 42)
Test Shape: (100000, 42)


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 43 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   Available Extra Rooms in Hospital      500000 non-null  int64  
 1   staff_available                        500000 non-null  int64  
 2   Visitors with Patient                  500000 non-null  int64  
 3   Admission_Deposit                      500000 non-null  float64
 4   Stay (in days)                         500000 non-null  int64  
 5   Department_anesthesia                  500000 non-null  uint8  
 6   Department_gynecology                  500000 non-null  uint8  
 7   Department_radiotherapy                500000 non-null  uint8  
 8   Department_surgery                     500000 non-null  uint8  
 9   Ward_Facility_Code_B                   500000 non-null  uint8  
 10  Ward_Facility_Code_C                   500000 non-null  

# Model Building

In [ ]:
# Dropping the specified features
x_train2 = x_train1.drop(columns = {'staff_available', 'Visitors with Patient', 'Ward_Facility_Code_F', 'Age_81-90', 'gender_Other', 'health_conditions_High Blood Pressure', 'health_conditions_None', 'health_conditions_Other', 'Insurance_Yes'})

x_test2 = x_test1.drop(columns = {'staff_available', 'Visitors with Patient', 'Ward_Facility_Code_F', 'Age_81-90', 'gender_Other', 'health_conditions_High Blood Pressure', 'health_conditions_None', 'health_conditions_Other', 'Insurance_Yes'})



In [23]:
from sklearn.model_selection import cross_val_score

# Build the regression model using sklearn linear regression
linreg = LinearRegression()                                    
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2) Score:", r2)

Mean Squared Error (MSE): 9.884965445902619
Mean Absolute Error (MAE): 2.155759034562716
R-squared (R2) Score: 0.8430298956456241


# Random Forest Regressor

In [25]:
# Random Forest Regressor
regressor = RandomForestRegressor(n_estimators = 100, random_state = 1)

# Fitting the model
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2) Score:", r2)

Mean Squared Error (MSE): 1.6995923960000001
Mean Absolute Error (MAE): 0.8650502000000002
R-squared (R2) Score: 0.9730110138249792


In [26]:
#fitting the gradient boost regressor 
from sklearn.ensemble import GradientBoostingRegressor

GBreg = GradientBoostingRegressor(n_estimators = 100, random_state = 1, learning_rate=0.1, max_depth=3)

GBreg.fit(X_train, y_train)

y_pred = GBreg.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2) Score:", r2)


Mean Squared Error (MSE): 3.213847966912557
Mean Absolute Error (MAE): 1.2127491554423462
R-squared (R2) Score: 0.9489651174294724


In [28]:
#fitting XGBoost
import xgboost as xgb

xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=3, random_state=1)
#fitting the model
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2) Score:", r2)


Mean Squared Error (MSE): 3.2179812213485643
Mean Absolute Error (MAE): 1.2135936201810837
R-squared (R2) Score: 0.9488994826648702


In [29]:
#fitting lightGBM
import lightgbm as lgb

# Define the dataset format for LightGBM
train_data = lgb.Dataset(X_train, label=y_train)

# Set the hyperparameters for the model
params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'metric': 'mae',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'random_state': 1
}
#training the model
lgb_model = lgb.train(params, train_data, num_boost_round=100)


y_pred = lgb_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2) Score:", r2)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026965 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 388
[LightGBM] [Info] Number of data points in the train set: 400000, number of used features: 42
[LightGBM] [Info] Start training from score 12.375650
Mean Squared Error (MSE): 2.505736080127015
Mean Absolute Error (MAE): 1.0928246579135998
R-squared (R2) Score: 0.9602097087607829


In [36]:
#hyperparameter tuning using grid search
rf_tuned = RandomForestRegressor(n_estimators = 100,random_state = 1)

# Define the hyperparameters to be tuned
param_grid = {
    
    "max_depth": [5, 7],
    "max_features": [0.8, 1]
}


# Type of scoring used to compare parameter combinations
scorer = make_scorer(r2_score)

# Run the grid search
grid_obj = GridSearchCV(rf_tuned, param_grid, cv = 5)

grid_obj = grid_obj.fit(X_train, y_train)

# Set the rf_tuned_regressor to the best combination of parameters
rf_tuned_regressor = grid_obj.best_estimator_

rf_tuned_regressor.fit(X_train, y_train)

y_pred = rf_tuned_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2) Score:", r2)

Mean Squared Error (MSE): 2.9687492649150133
Mean Absolute Error (MAE): 1.1564083439719384
R-squared (R2) Score: 0.9528572067888362


# Neural Network

In [31]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

#define the model
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               5504      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 256)               65792     
                                                                 
 dense_4 (Dense)             (None, 1)                 257       
                                                                 
Total params: 170,369
Trainable params: 170,369
Non-trainable params: 0
_________________________________________________________________


In [32]:
checkpoint_name = 'NN_model.hdf5'
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [33]:
NN_model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

Epoch 1/30
 9989/10000 [============================>.] - ETA: 0s - loss: 2.5214 - mean_absolute_error: 2.5214
Epoch 1: val_loss improved from inf to 1.50389, saving model to NN_model.hdf5
10000/10000 [==============================] - 54s 5ms/step - loss: 2.5205 - mean_absolute_error: 2.5205 - val_loss: 1.5039 - val_mean_absolute_error: 1.5039
Epoch 2/30
 9992/10000 [============================>.] - ETA: 0s - loss: 1.6511 - mean_absolute_error: 1.6511
Epoch 2: val_loss did not improve from 1.50389
10000/10000 [==============================] - 56s 6ms/step - loss: 1.6510 - mean_absolute_error: 1.6510 - val_loss: 1.5102 - val_mean_absolute_error: 1.5102
Epoch 3/30
 9999/10000 [============================>.] - ETA: 0s - loss: 1.5911 - mean_absolute_error: 1.5911
Epoch 3: val_loss improved from 1.50389 to 1.45216, saving model to NN_model.hdf5
10000/10000 [==============================] - 51s 5ms/step - loss: 1.5911 - mean_absolute_error: 1.5911 - val_loss: 1.4522 - val_mean_absolute_

In [34]:

y_pred = NN_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2) Score:", r2)

3125/3125 [==============================] - 5s 2ms/step
Mean Squared Error (MSE): 4.125034535961448
Mean Absolute Error (MAE): 1.414161374759674
R-squared (R2) Score: 0.9344957647936272


The RandomForestRegressor still has the best performance. with the lowest NSE and highest Rsquared

In [54]:
!pip install  -q fastapi nest-asyncio pyngrok uvicorn


In [44]:
import joblib

joblib.dump(regressor, 'LOSmodel.pkl')


['LOSmodel.pkl']

In [62]:
from google.colab import files
files.download('LOSmodel.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [55]:
%%writefile app.py

from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import pandas as pd

class InputData(BaseModel):
    Available_Extra_Rooms_in_Hospital: int
    Department: str
    Ward_Facility_Code: str
    doctor_name: str
    staff_available: int
    #patientid: int
    Age: str
    gender: str
    Type_of_Admission: str
    Severity_of_Illness: str
    health_conditions: str
    Visitors_with_Patient: int
    Insurance: str
    Admission_Deposit: float

app = FastAPI()

model= joblib.load('/content/LOSmodel.pkl')

def predict(input_data: InputData):
    # Convert the input data to a dictionary
    input_dict = input_data.dict()

    # Create a DataFrame from the input dictionary
    input_df = pd.DataFrame([input_dict])

    # Perform one-hot encoding on the categorical features
    categorical_features = ['Department', 'Ward_Facility_Code', 'doctor_name', 'Age', 'gender',
                            'Type_of_Admission', 'Severity_of_Illness', 'health_conditions', 'Insurance']
    encoded_features = pd.get_dummies(input_df[categorical_features], drop_first=True)

    processed_features = pd.concat([encoded_features, input_df.drop(categorical_features, axis=1)], axis=1)

    predictions = model.predict(processed_features)

    return {"predictions": predictions.tolist()}

Overwriting app.py


In [57]:
#!uvicorn app:app --port 8000
!uvicorn app:app --host 0.0.0.0 --port 80ccc

Public URL: https://c89b-35-247-98-223.ngrok.io


NameError: ignored

In [50]:
!pip install -q pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [51]:
from pyngrok import ngrok

public_url = ngrok.connect(port=8000)
public_url


PyngrokNgrokHTTPError: ignored